### Añadimos SRC a la raíz del proyecto para poder importar el contenido

In [ ]:
import sys
import os

# Obtener la ruta absoluta de la carpeta raíz (donde está src)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Subir un nivel desde notebooks/

# Agregar la carpeta src al path
sys.path.append(os.path.join(ROOT_DIR, "src"))

### Importamos los módulos necesarios

In [ ]:
import pandas as pd

from config import SMOGON_SCRAPPING_DATA_PATH, POKEAPI_FINAL_DATASET_PATH, POKEAPI_MISSING_IN_VGC_SMOGON_PATH, VGC_SMOGON_MISSING_IN_POKEAPI_PATH, POKEAPI_DATASET_PATH_WITH_SMOGON

### Merge con el dataset PokeAPI

In [ ]:
# 📂 Cargar los datasets
df_vgc = pd.read_csv(SMOGON_SCRAPPING_DATA_PATH)  # Dataset con datos de uso VGC
df_main = pd.read_csv(POKEAPI_FINAL_DATASET_PATH)  # Dataset principal de PokeAPI

# 🔎 Revisar valores nulos antes de limpiar
print("🔎 Valores nulos en df_vgc antes de limpieza:", df_vgc["Pokemon"].isna().sum())
print("🔎 Valores nulos en df_main antes de limpieza:", df_main["name"].isna().sum())

# 📌 LIMPIEZA DE NOMBRES EN df_vgc
df_vgc["Pokemon"] = df_vgc["Pokemon"].fillna("")  # Asegurar que no haya valores nulos
df_vgc["Pokemon"] = df_vgc["Pokemon"].astype(str).str.lower()
df_vgc["Pokemon"] = df_vgc["Pokemon"].str.replace(r"\.", "", regex=True)  # Eliminar "."
df_vgc["Pokemon"] = df_vgc["Pokemon"].str.replace(r"'", "", regex=True)   # Eliminar "'"
df_vgc["Pokemon"] = df_vgc["Pokemon"].str.replace(r"\s+", "-", regex=True)  # Reemplazar espacios con "-"
df_vgc["Pokemon"] = df_vgc["Pokemon"].str.strip()  # Eliminar espacios sobrantes

# 📌 LIMPIEZA DE NOMBRES EN df_main
df_main["name"] = df_main["name"].fillna("")  # Asegurar que no haya valores nulos
df_main["name"] = df_main["name"].astype(str).str.lower()
df_main["name"] = df_main["name"].str.replace(r"\.", "", regex=True)
df_main["name"] = df_main["name"].str.replace(r"'", "", regex=True)
df_main["name"] = df_main["name"].str.replace(r"\s+", "-", regex=True)
df_main["name"] = df_main["name"].str.strip()

# 🔎 Verificar nombres después de limpieza
print("\n🔎 Ejemplo de nombres en df_vgc después de limpieza:", df_vgc["Pokemon"].head(10).tolist())
print("🔎 Ejemplo de nombres en df_main después de limpieza:", df_main["name"].head(10).tolist())


### Eliminación de Silvally ya que es un dato conflictivo

In [ ]:
# 🔎 Mostrar cuántas filas contienen "silvally"
silvally_count = df_vgc["Pokemon"].str.contains("type:-null", case=False, na=False).sum()
print(f"⚠️ Filas con 'silvally' antes de la limpieza: {silvally_count}")

# 🗑️ Eliminar filas donde "Pokemon" contenga "silvally"
df_vgc_filtered = df_vgc[~df_vgc["Pokemon"].str.contains("type:-null", case=False, na=False)]

# 🔎 Mostrar cuántas filas quedan después del filtrado
print(f"✅ Filas restantes después de eliminar 'type:-null': {len(df_vgc_filtered)}")

# 📁 Guardar el dataset limpio
df_vgc_filtered.to_csv(SMOGON_SCRAPPING_DATA_PATH, index=False)

print(f"📁 Dataset limpio guardado en '{output_path}'")

### Comparación de coincidencia de nombres entre datasets de PokeAPI y Smogon

In [ ]:
# 🔎 Revisar valores nulos antes de limpiar
print("🔎 Valores nulos en df_vgc antes de limpieza:", df_vgc["Pokemon"].isna().sum())
print("🔎 Valores nulos en df_main antes de limpieza:", df_main["name"].isna().sum())

# 📌 LIMPIEZA DE NOMBRES EN df_vgc
df_vgc["Pokemon"] = df_vgc["Pokemon"].fillna("")  # Asegurar que no haya valores nulos
df_vgc["Pokemon"] = df_vgc["Pokemon"].astype(str).str.lower()
df_vgc["Pokemon"] = df_vgc["Pokemon"].str.replace(r"\.", "", regex=True)  # Eliminar "."
df_vgc["Pokemon"] = df_vgc["Pokemon"].str.replace(r"'", "", regex=True)   # Eliminar "'"
df_vgc["Pokemon"] = df_vgc["Pokemon"].str.replace(r"\s+", "-", regex=True)  # Reemplazar espacios con "-"
df_vgc["Pokemon"] = df_vgc["Pokemon"].str.strip()  # Eliminar espacios sobrantes

# 📌 LIMPIEZA DE NOMBRES EN df_main
df_main["name"] = df_main["name"].fillna("")  # Asegurar que no haya valores nulos
df_main["name"] = df_main["name"].astype(str).str.lower()
df_main["name"] = df_main["name"].str.replace(r"\.", "", regex=True)
df_main["name"] = df_main["name"].str.replace(r"'", "", regex=True)
df_main["name"] = df_main["name"].str.replace(r"\s+", "-", regex=True)
df_main["name"] = df_main["name"].str.strip()

# 🔎 Verificar nombres después de limpieza
print("\n🔎 Ejemplo de nombres en df_vgc después de limpieza:", df_vgc["Pokemon"].head(10).tolist())
print("🔎 Ejemplo de nombres en df_main después de limpieza:", df_main["name"].head(10).tolist())

# 📌 3. COMPARACIÓN ENTRE LOS DOS DATASETS
vgc_names = set(df_vgc["Pokemon"])  # Nombres en VGC
main_names = set(df_main["name"])  # Nombres en el dataset principal

# ✅ Coincidencias
matched_names = vgc_names & main_names  # Nombres que coinciden
unmatched_vgc = vgc_names - main_names  # Nombres en VGC que no están en el dataset principal
unmatched_main = main_names - vgc_names  # Nombres en el dataset principal que no están en VGC

# 📊 Mostrar estadísticas
print(f"🔍 Total de nombres en VGC: {len(vgc_names)}")
print(f"🔍 Total de nombres en el dataset principal: {len(main_names)}")
print(f"✅ Coincidencias entre ambos datasets: {len(matched_names)}")
print(f"⚠️ Pokémon en VGC que no están en el dataset principal: {len(unmatched_vgc)}")
print(f"⚠️ Pokémon en el dataset principal que no están en VGC: {len(unmatched_main)}")

# 📁 Guardar listas de Pokémon huérfanos para revisión
pd.DataFrame({"Unmatched_VGC": list(unmatched_vgc)}).to_csv(VGC_SMOGON_MISSING_IN_POKEAPI_PATH, index=False)
pd.DataFrame({"Unmatched_Main": list(unmatched_main)}).to_csv(POKEAPI_MISSING_IN_VGC_SMOGON_PATH, index=False)

print("📁 Listado de nombres huérfanos guardado en '{VGC_SMOGON_MISSING_IN_POKEAPI_PATH} y '{POKEAPI_MISSING_IN_VGC_SMOGON_PATH}'.")

### Una vez tenemos todos los nombres, hacemos el Merge

In [ ]:
# 📂 Cargar los datasets
df_main = pd.read_csv(POKEAPI_FINAL_DATASET_PATH)  # Dataset principal con todos los Pokémon
df_vgc = pd.read_csv(SMOGON_SCRAPPING_DATA_PATH)  # Dataset de VGC ya limpio

# 🔎 Mostrar las columnas de ambos datasets
print(f"📊 Columnas en df_main: {df_main.columns.tolist()}")
print(f"📊 Columnas en df_vgc: {df_vgc.columns.tolist()}")

# 🔗 Merge de los datasets (Unión por el nombre del Pokémon)
df_merged = df_main.merge(df_vgc, left_on="name", right_on="Pokemon", how="left")

# 🗑️ Eliminar la columna duplicada "Pokemon" después del merge
df_merged.drop(columns=["Pokemon"], inplace=True)

# 🛠️ Rellenar valores NaN en las columnas de uso de VGC con "NoUsage"
vgc_cols = ["VGCusage2022", "VGCusage2023", "VGCusage2024"]  # Columnas de uso de VGC
df_merged[vgc_cols] = df_merged[vgc_cols].fillna("NoUsage")

# 📊 Revisar cuántos Pokémon no tenían datos de VGC
no_usage_count = (df_merged[vgc_cols] == "NoUsage").sum().sum()
print(f"⚠️ Pokémon sin datos de VGC rellenados con 'NoUsage': {no_usage_count}")

# 📁 Guardar el dataset final con los datos de VGC integrados
df_merged.to_csv(POKEAPI_DATASET_PATH_WITH_SMOGON, index=False)

print(f"✅ Dataset final guardado en '{POKEAPI_DATASET_PATH_WITH_SMOGON}'")

In [ ]:
df_merged.info()

### Rellenamos los valores nulos con valores "No_type" y "No_ability"

In [ ]:
# 📂 Cargar el dataset con VGC integrado
df_merged = pd.read_csv(POKEAPI_DATASET_PATH_WITH_SMOGON)

# 🛠️ Rellenar valores nulos en Type2 y ability2 con "None"
df_merged["type2"] = df_merged["type2"].fillna("No_type")
df_merged["ability2"] = df_merged["ability2"].fillna("No_ability")

# 📊 Verificar que no haya valores nulos restantes
missing_values = df_merged[["type2", "ability2"]].isnull().sum()
print(f"🔎 Valores nulos restantes después del relleno:\n{missing_values}")

# 📁 Guardar el dataset final
df_merged.to_csv(POKEAPI_DATASET_PATH_WITH_SMOGON, index=False)

print(f"✅ Dataset final guardado en '{POKEAPI_DATASET_PATH_WITH_SMOGON}'")

In [ ]:
df_merged.info()